In [1]:
!pip install nltk textblob transformers torch pyyaml


In [2]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
import os

folders = ["tool_agent", "tool_agent/logs"]
for f in folders:
    os.makedirs(f, exist_ok=True)


In [25]:
%%writefile tool_agent/config.yaml
constraints:
  speed_weight: 0.35
  accuracy_weight: 0.20
  resource_weight: 0.25


Overwriting tool_agent/config.yaml


In [5]:
%%writefile tool_agent/tool_registry.py

TOOLS = {
    "vader": {
        "speed": 9,
        "accuracy": 6,
        "resource": 9,
        "offline": True,
        "description": "NLTK VADER sentiment analyzer"
    },
    "textblob": {
        "speed": 7,
        "accuracy": 7,
        "resource": 8,
        "offline": True,
        "description": "TextBlob polarity analyzer"
    },
    "huggingface": {
        "speed": 5,
        "accuracy": 9,
        "resource": 4,
        "offline": False,
        "description": "Transformer sentiment model"
    }
}


Writing tool_agent/tool_registry.py


In [6]:
%%writefile tool_agent/decision_engine.py

import yaml

def load_weights():
    with open("tool_agent/config.yaml") as f:
        return yaml.safe_load(f)["constraints"]

def score_tool(meta, w):
    return meta["speed"]*w["speed_weight"] + meta["accuracy"]*w["accuracy_weight"] + meta["resource"]*w["resource_weight"]

def select_best_tool(tools):
    w = load_weights()
    scores = {k: score_tool(v, w) for k,v in tools.items()}
    return max(scores, key=scores.get), scores


Writing tool_agent/decision_engine.py


In [7]:
%%writefile tool_agent/executor.py

import os
import warnings
from contextlib import redirect_stdout, redirect_stderr
import io

# Silence all HF + system logs
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
warnings.filterwarnings("ignore")

from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from transformers import pipeline, logging as hf_logging

hf_logging.set_verbosity_error()

def silent(func, *args):
    f = io.StringIO()
    with redirect_stdout(f), redirect_stderr(f):
        return func(*args)

def run_vader(texts):
    sia = SentimentIntensityAnalyzer()
    return [sia.polarity_scores(t)["compound"] for t in texts]

def run_textblob(texts):
    return [TextBlob(t).sentiment.polarity for t in texts]

def run_huggingface(texts, batch=32):
    clf = silent(pipeline, "sentiment-analysis")
    results = []
    for i in range(0, len(texts), batch):
        results.extend(silent(clf, texts[i:i+batch]))
    return results

def execute(tool, data):
    return {
        "vader": run_vader,
        "textblob": run_textblob,
        "huggingface": run_huggingface
    }[tool](data)


Writing tool_agent/executor.py


In [8]:
%%writefile tool_agent/logger.py

from datetime import datetime

def log_decision(text):
    with open("tool_agent/logs/decisions.log", "a") as f:
        f.write(f"{datetime.now()} | {text}\n")


Writing tool_agent/logger.py


In [33]:
%%writefile tool_agent/agent.py

from tool_registry import TOOLS
from decision_engine import select_best_tool
from executor import execute, run_vader, run_textblob, run_huggingface
from logger import log_decision
import time

class ToolChoosingAgent:

    def runtime(self, fn, data):
        t=time.time(); fn(data); return round(time.time()-t,3)

    def accuracy(self, fn, data, labels):
        preds=fn(data)
        correct=0
        for p,l in zip(preds,labels):
            s=1 if (p['label']=='POSITIVE' if isinstance(p,dict) else p>0) else -1
            if s==l: correct+=1
        return round(correct/len(labels)*100,2)

    def run_task(self, task, data):

        eval_data=[("This product is amazing!",1),("Very bad experience",-1),
                   ("I love this service",1),("Worst purchase ever",-1)]

        texts=[x[0] for x in eval_data]
        labels=[x[1] for x in eval_data]

        best,scores=select_best_tool(TOOLS)

        log_decision(f"Task: {task}")
        log_decision(f"Scores: {scores}")
        log_decision(f"Chosen Tool: {best}")

        print("\n================ TOOL EVALUATION REPORT ================\n")
        for k in scores:
            print(f"{k.capitalize():12}: {scores[k]:.2f}")

        print("\nSelected Tool :", best.upper())
        print("\n================ PERFORMANCE METRICS ===================\n")
        print("Tool        Runtime(s)   Accuracy(%)")

        print(f"VADER        {self.runtime(run_vader,texts)}        {self.accuracy(run_vader,texts,labels)}")
        print(f"TextBlob     {self.runtime(run_textblob,texts)}        {self.accuracy(run_textblob,texts,labels)}")
        print(f"HuggingFace  {self.runtime(run_huggingface,texts)}        {self.accuracy(run_huggingface,texts,labels)}")

        print("\n================ EXECUTION RESULTS =====================\n")

        results=execute(best,data)
        for i,r in enumerate(results[:10],1):
            print(f"Review {i} : {r}")

        return results


Overwriting tool_agent/agent.py


In [10]:
import random
import os

# Make sure folder exists
os.makedirs("tool_agent/data", exist_ok=True)

positive = [
    "This product is amazing and works perfectly",
    "Excellent quality and very useful",
    "I am extremely satisfied with this purchase",
    "Great value for money",
    "Highly recommend this product",
    "Fast delivery and good packaging",
    "Loved the design and performance",
    "Works exactly as described",
    "Very happy with the service",
    "Best purchase I made this year"
]

negative = [
    "Very bad experience overall",
    "Totally disappointed with this product",
    "Waste of money",
    "Stopped working after a few days",
    "Poor quality material",
    "Terrible customer support",
    "Not worth the price",
    "Product arrived damaged",
    "Worst purchase ever",
    "Would not recommend this"
]

neutral = [
    "The product is okay for the price",
    "Average quality nothing special",
    "It works but could be better",
    "Not bad but not great either",
    "Satisfactory performance",
    "Delivery was fine",
    "Decent product overall",
    "It does the job",
    "Neither good nor bad",
    "Acceptable quality"
]

mixed = [
    "Good product but delivery was late",
    "Quality is good but price is high",
    "Works well but customer service is poor",
    "Nice design but average performance",
    "Fast delivery but packaging was bad",
    "Useful but slightly expensive",
    "Looks good but not very durable",
    "Good value but could be improved",
    "Performance is fine but build quality is weak",
    "Helpful product but instructions were unclear"
]

all_reviews = positive + negative + neutral + mixed

dataset = [random.choice(all_reviews) for _ in range(5000)]

file_path = "tool_agent/data/sample_5000_reviews.txt"

with open(file_path, "w") as f:
    f.write("\n".join(dataset))

print("Created file at:", file_path)
print("Total reviews:", len(dataset))
print("\nFolder contents:", os.listdir("tool_agent/data"))


Created file at: tool_agent/data/sample_5000_reviews.txt
Total reviews: 5000

Folder contents: ['sample_5000_reviews.txt']


In [11]:
%%writefile tool_agent/main.py

from agent import ToolChoosingAgent

def load_reviews(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read().splitlines()

if __name__ == "__main__":

    reviews = load_reviews("tool_agent/data/sample_5000_reviews.txt")

    print(f"\nLoaded {len(reviews)} user reviews for analysis\n")

    agent = ToolChoosingAgent()
    agent.run_task("Sentiment Analysis on 5,000 Reviews", reviews)


Writing tool_agent/main.py


In [34]:
!python tool_agent/main.py



Loaded 5000 user reviews for analysis


================ TOOL EVALUATION REPORT ================

Vader       : 6.60
Textblob    : 5.85
Huggingface : 4.55

Selected Tool : VADER

================ PERFORMANCE METRICS ===================

Tool        Runtime(s)   Accuracy(%)
VADER        0.017        100.0
TextBlob     0.045        100.0
HuggingFace  1.083        100.0

================ EXECUTION RESULTS =====================

Review 1 : 0.0
Review 2 : 0.6369
Review 3 : 0.7783
Review 4 : 0.0
Review 5 : 0.2382
Review 6 : -0.1027
Review 7 : 0.3182
Review 8 : 0.8402
Review 9 : 0.3182
Review 10 : -0.2263


# **Decision Insight**
- **VADER** is fastest and resource-efficient.
- **HuggingFace** is most accurate but slowest.
- **Agent selects** based on constraint priorities.

In [35]:
!cat tool_agent/logs/decisions.log


2026-02-04 07:04:07.959834 | Task: Sentiment Analysis on 5,000 Reviews
2026-02-04 07:04:07.959981 | Scores: {'vader': 8.1, 'textblob': 7.35, 'huggingface': 5.85}
2026-02-04 07:04:07.960031 | Chosen Tool: vader
2026-02-04 07:04:34.512910 | Task: Sentiment Analysis on 5,000 Reviews
2026-02-04 07:04:34.512999 | Scores: {'vader': 8.1, 'textblob': 7.35, 'huggingface': 5.85}
2026-02-04 07:04:34.513029 | Chosen Tool: vader
2026-02-04 07:05:28.900113 | Task: Sentiment Analysis on 5,000 Reviews
2026-02-04 07:05:28.900210 | Scores: {'vader': 4.35, 'textblob': 4.1, 'huggingface': 3.3}
2026-02-04 07:05:28.900238 | Chosen Tool: vader
2026-02-04 07:06:05.810521 | Task: Sentiment Analysis on 5,000 Reviews
2026-02-04 07:06:05.810610 | Scores: {'vader': 6.6, 'textblob': 5.85, 'huggingface': 4.55}
2026-02-04 07:06:05.810638 | Chosen Tool: vader
2026-02-04 07:06:41.213834 | Task: Sentiment Analysis on 5,000 Reviews
2026-02-04 07:06:41.213931 | Scores: {'vader': 6.6, 'textblob': 5.85, 'huggingface': 4.55}

In [36]:
!zip -r tool_agent.zip tool_agent


updating: tool_agent/ (stored 0%)
updating: tool_agent/agent.py (deflated 59%)
updating: tool_agent/__pycache__/ (stored 0%)
updating: tool_agent/__pycache__/tool_registry.cpython-312.pyc (deflated 23%)
updating: tool_agent/__pycache__/executor.cpython-312.pyc (deflated 39%)
updating: tool_agent/__pycache__/logger.cpython-312.pyc (deflated 28%)
updating: tool_agent/__pycache__/decision_engine.cpython-312.pyc (deflated 34%)
updating: tool_agent/__pycache__/agent.cpython-312.pyc (deflated 43%)
updating: tool_agent/tool_registry.py (deflated 61%)
updating: tool_agent/executor.py (deflated 52%)
updating: tool_agent/.ipynb_checkpoints/ (stored 0%)
updating: tool_agent/main.py (deflated 38%)
updating: tool_agent/logs/ (stored 0%)
updating: tool_agent/logs/decisions.log (deflated 77%)
updating: tool_agent/decision_engine.py (deflated 46%)
updating: tool_agent/data/ (stored 0%)
updating: tool_agent/data/sample_5000_reviews.txt (deflated 93%)
updating: tool_agent/logger.py (deflated 26%)
updati

In [37]:
from google.colab import files
files.download("tool_agent.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
%%capture output
!python tool_agent/main.py


In [39]:
with open("README.md", "w") as f:
    f.write("# Tool-Chosing Autonomous Agent – Output Report\n\n")
    f.write("```\n")
    f.write(output.stdout)
    f.write("\n```")


In [40]:
from google.colab import files
files.download("README.md")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>